In [118]:
#libraries we need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import date
pd.options.mode.chained_assignment = None
import h2o
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

#libraries we need
# !pip install h2o

from scipy.special import expit
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.grid.grid_search import H2OGridSearch
from sklearn import preprocessing

from sklearn.model_selection import train_test_split
from h2o.estimators import H2OGradientBoostingEstimator
SEED  = 1111   # global random seed for better reproducibility

from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus

h2o.init(max_mem_size='24G', nthreads=4) # start h2o with plenty of memory and threads
h2o.remove_all()                         # clears h2o memory
h2o.no_progress() 

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,18 mins 24 secs
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.5
H2O_cluster_version_age:,"14 days, 22 hours and 42 minutes"
H2O_cluster_name:,H2O_from_python_vibhz_bnxw93
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,21.21 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [119]:
train = pd.read_csv('adult.data.txt',
                    names=['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','class'])

In [120]:
test = pd.read_csv('adult.test.txt',
                    names=['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','class'])

In [121]:
train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [122]:
#train.to_csv('train.csv',index=False)

In [123]:
#test.to_csv('test.csv',index=False)

In [124]:
train.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
class             object
dtype: object

In [125]:
train.shape

(32561, 15)

In [126]:
test_y = test['class']
test_x = test.drop(['class'],axis=1)

full = train.append(test_x, ignore_index=True)

In [127]:
cc = [key for key in dict(train.dtypes) if dict(train.dtypes)[key] in ['float64', 'int64']]
obj = [key for key in dict(train.dtypes) if dict(train.dtypes)[key] in ['object']]

for i in obj:
    print(full[i].value_counts())
    



 Private             33906
 Self-emp-not-inc     3862
 Local-gov            3136
 ?                    2799
 State-gov            1981
 Self-emp-inc         1695
 Federal-gov          1432
 Without-pay            21
 Never-worked           10
Name: workclass, dtype: int64
 HS-grad         15784
 Some-college    10878
 Bachelors        8025
 Masters          2657
 Assoc-voc        2061
 11th             1812
 Assoc-acdm       1601
 10th             1389
 7th-8th           955
 Prof-school       834
 9th               756
 12th              657
 Doctorate         594
 5th-6th           509
 1st-4th           247
 Preschool          83
Name: education, dtype: int64
 Married-civ-spouse       22379
 Never-married            16117
 Divorced                  6633
 Separated                 1530
 Widowed                   1518
 Married-spouse-absent      628
 Married-AF-spouse           37
Name: marital-status, dtype: int64
 Prof-specialty       6172
 Craft-repair         6112
 Exec-managerial

In [128]:
## USE THE ABOVE TO REMOVE THE '?'

In [129]:
for i in obj:
    print(full[i].unique())

[' State-gov' ' Self-emp-not-inc' ' Private' ' Federal-gov' ' Local-gov'
 ' ?' ' Self-emp-inc' ' Without-pay' ' Never-worked']
[' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' Assoc-voc' ' 7th-8th' ' Doctorate' ' Prof-school'
 ' 5th-6th' ' 10th' ' 1st-4th' ' Preschool' ' 12th']
[' Never-married' ' Married-civ-spouse' ' Divorced'
 ' Married-spouse-absent' ' Separated' ' Married-AF-spouse' ' Widowed']
[' Adm-clerical' ' Exec-managerial' ' Handlers-cleaners' ' Prof-specialty'
 ' Other-service' ' Sales' ' Craft-repair' ' Transport-moving'
 ' Farming-fishing' ' Machine-op-inspct' ' Tech-support' ' ?'
 ' Protective-serv' ' Armed-Forces' ' Priv-house-serv']
[' Not-in-family' ' Husband' ' Wife' ' Own-child' ' Unmarried'
 ' Other-relative']
[' White' ' Black' ' Asian-Pac-Islander' ' Amer-Indian-Eskimo' ' Other']
[' Male' ' Female']
[' United-States' ' Cuba' ' Jamaica' ' India' ' ?' ' Mexico' ' South'
 ' Puerto-Rico' ' Honduras' ' England' ' Canada' ' Germany' 

In [130]:
full=full.replace(' ?', np.nan)

In [131]:
full.isna().sum()

age                   0
capital-gain          0
capital-loss          0
class             16281
education             0
education-num         0
fnlwgt                0
hours-per-week        0
marital-status        0
native-country      857
occupation         2809
race                  0
relationship          0
sex                   0
workclass          2799
dtype: int64

In [132]:
full.update(full[['native-country','occupation','workclass']].fillna('None'))

train = full[full['class'].notnull()]
test_x = full[full['class'].isnull()].drop(['class'],axis=1)




In [133]:
## GLM

In [134]:
train_y = train['class']
train_x = train.drop(['class'],axis=1)

In [149]:
le = preprocessing.LabelEncoder()

train_df_x = pd.get_dummies(train_x)
test_df_x = pd.get_dummies(test_x)
#train_df_y = pd.DataFrame(le.fit_transform(train_y))
#test_df_y = pd.DataFrame(le.fit_transform(test_y))


train_data = pd.concat([train_y, train_x], axis=1)
train_data.rename({'0': 'class'}, axis=1, inplace=True)
test_df_y.rename({'0': 'class'}, axis=1, inplace=True)




In [150]:
hf=h2o.H2OFrame(train_data)
gf=h2o.H2OFrame(test_x)

In [151]:
train_data.shape

(32561, 15)

In [152]:
test_x.shape

(16281, 14)

In [153]:
r = 'class'
x = list(train_data.columns.values)

In [154]:
hyper_params = {'alpha': [0, .25, .5, .75, 1]
                ,'lambda':[1, 0.5, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0]
               }

glm = H2OGeneralizedLinearEstimator(family = 'binomial',standardize = True,lambda_search = True)

# build grid search with previously made GLM and hyperparameters
grid = H2OGridSearch(model = glm, hyper_params = hyper_params,
                     search_criteria = {'strategy': "Cartesian"})


grid.train(x = x, y = r, training_frame = hf,nfolds=5,seed=1)

In [155]:
sorted_grid = grid.get_grid(sort_by='accuracy', decreasing=False)
best_glm = sorted_grid.models[0]
best_glm.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.24081142,0.0055863853,0.2443321,0.2405397,0.23948562,0.24722393,0.23247574
1,auc,0.5,0.0,0.5,0.5,0.5,0.5,0.5
2,aucpr,0.24081142,0.0055863853,0.2443321,0.2405397,0.23948562,0.24722393,0.23247574
3,err,0.7591886,0.0055863853,0.75566787,0.7594603,0.7605144,0.7527761,0.76752424
4,err_count,4944.0,60.481403,4933.0,4897.0,5027.0,4881.0,4982.0
5,f0point5,0.28391483,0.006215691,0.2878334,0.2836192,0.28244656,0.2910418,0.2746333
6,f1,0.38812533,0.0072700256,0.39271206,0.3877985,0.38642743,0.39643872,0.37725
7,f2,0.613233,0.0072813053,0.6178339,0.61294657,0.6115747,0.62151057,0.60229903
8,lift_top_group,1.0,0.0,1.0,1.0,1.0,1.0,1.0
9,logloss,0.5520517,0.0063958997,0.5560752,0.55170155,0.55049384,0.5594395,0.5425484


In [156]:
#best_glm

In [157]:
pred_glm_tr =  best_glm.predict(hf)
pred_glm_tr = pred_glm_tr.as_data_frame()
co = best_glm.coef()

In [159]:
pred_glm_tr.head()

,predict,<=50K,>50K
0,<=50K,0.75919,0.24081
1,<=50K,0.75919,0.24081
2,<=50K,0.75919,0.24081
3,<=50K,0.75919,0.24081
4,<=50K,0.75919,0.24081
